In [ ]:
import math
import random
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from sklearn import datasets
from sklearn import model_selection
from sklearn import metrics
from sklearn import linear_model
import pickle
import lightgbm
import ast
import csv
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import accuracy_score

In [2]:
data = pd.read_csv('../data/onti_c.csv')

In [ ]:
!conda install networkx

Solving environment: done

## Package Plan ##

  environment location: /home/ec2-user/anaconda3

  added / updated specs:
    - networkx


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    networkx-2.5               |             py_0         1.1 MB
    ------------------------------------------------------------
                                           Total:         1.1 MB

The following NEW packages will be INSTALLED:

  networkx           pkgs/main/noarch::networkx-2.5-py_0


Proceed ([y]/n)? 

In [3]:
import hyperopt

def hyperopt_objective(params):
    model = CatBoostClassifier(
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        iterations=10,
        eval_metric='Accuracy',
        random_seed=42,
        verbose=False,
        loss_function='Logloss',
        od_type='Iter',
        od_wait=20
    )
    
    cv_data = cv(
        Pool(x, y, cat_features=categorical_features_indices),
        model.get_params()
    )
    best_accuracy = np.max(cv_data['test-Accuracy-mean'])
    
    return 1 - best_accuracy # as hyperopt minimises

ModuleNotFoundError: No module named 'networkx'

In [ ]:
from numpy.random import RandomState

params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1)
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=5,
    trials=trials,
    rstate=RandomState(123)
)

print(best)

In [ ]:
model = CatBoostClassifier(
    l2_leaf_reg=int(best['l2_leaf_reg']),
    learning_rate=best['learning_rate'],
    iterations=500,
    eval_metric='Accuracy',
    random_seed=42,
    verbose=True,
    loss_function='Logloss',
)
cv_data = cv(Pool(x, y, cat_features=categorical_features_indices), model.get_params())

In [ ]:
print('Precise validation accuracy score: {}'.format(np.max(cv_data['test-Accuracy-mean'])))

In [ ]:
with open("catboost_flights.pkl","wb") as f:
    pickle.dump(model, f)